# House Prices in Wales

<div class="alert alert-block alert-info" style="margin-top: 20px; color: black;">
The aim of this project is to look at the price of properties in Wales, and see what useful conclusions can be drawn for people interested in buying Welsh property.Property sales data will be obtained from the UK Land Registry and combined with extra location data using the postcodes, in order to get the final dataset that will be used as the basis for this project.
</div>

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Import visualisation modules
from geopy.geocoders import Nominatim
import folium
import matplotlib as mpl
import matplotlib.pyplot as plt

# Import clustering algorithm from sci-kit learn
from sklearn.cluster import KMeans

## Load data

Load original data from HM Land Registry data © Crown copyright and database right 2020.
This data is licensed under the Open Government Licence v3.0 and can be found <a href='https://www.gov.uk/government/statistical-data-sets/price-paid-data-downloads'>here</a>.

In [ ]:
#header = ['Transaction ID', 'Price', 'Completion date', 'Postcode', 'Property type', 'New', 'Freehold', 'House no./name', 'Flat No', 'Street', 'Locality', 'Town/city', 'District', 'County']
#house_prices_2019 = pd.read_csv('../input/house-prices-england-and-wales/pp-2019.csv', names = header, index_col = False)
#house_prices_2019.drop(['Transaction ID', 'Completion date'], axis = 1, inplace = True)
#house_prices_2019['Property type'].replace({'S': 'Semi-detached', 'T': 'Terraced', 'D': 'Detached', 'F': 'Flats', 'O': 'Other'}, inplace = True)
#house_prices_2019.head()

In [ ]:
#header = ['Transaction ID', 'Price', 'Completion date', 'Postcode', 'Property type', 'New', 'Freehold', 'House no./name', 'Flat No', 'Street', 'Locality', 'Town/city', 'District', 'County']
#house_prices_2020 = pd.read_csv('../input/house-prices-england-and-wales/pp-2020.csv', names = header, index_col = False)
#house_prices_2020.drop(['Transaction ID', 'Completion date'], axis = 1, inplace = True)
#house_prices_2020['Property type'].replace({'S': 'Semi-detached', 'T': 'Terraced', 'D': 'Detached', 'F': 'Flats', 'O': 'Other'}, inplace = True)
#house_prices_2020.head()

Original postcode data downloaded from <a href='https://www.getthedata.com/open-postcode-geo'>getthedata.com</a>.   
Contains OS data © Crown copyright and database right 2021   
Contains Royal Mail data © Royal Mail copyright and database right 2021   
Contains National Statistics data © Crown copyright and database right 2021

In [ ]:
#header = ['postcode', 'status', 'usertype', 'easting', 'northing', 'positional quality indicator', 'country', 'latitude', 'longitude', 'postcode no space', 'postcode fixed width 7', 'postcode fixed width 8', 'postcode area', 'postcode district', 'postcode sector', 'outcode', 'incode']
#postcodes = pd.read_csv('../input/house-prices-england-and-wales/open_postcode_geo.csv', names = header, index_col = False)
#postcodes.drop(['status', 'usertype', 'easting', 'northing', 'positional quality indicator', 'postcode no space', 'postcode fixed width 7', 'postcode fixed width 8', 'postcode area', 'postcode district', 'postcode sector', 'outcode', 'incode'], axis = 1, inplace = True)
#postcodes.head()

## Combine datasets to form the final working dataframe

In [ ]:
"""
Add country, latitude and longitude columns to house_price dataframes.
"""
#postcodeList = list(postcodes['postcode'])
#countryList = list(postcodes['country'])
#latitudeList = list(postcodes['latitude'])
#longitudeList = list(postcodes['longitude'])

#country = []
#latitude = []
#longitude = []
#for postcode in house_prices_2019['Postcode']:
    #try:
        #location = postcodeList.index(postcode)
        #country.append(postcodes.at[location, 'country'])
        #latitude.append(postcodes.at[location, 'latitude'])
        #longitude.append(postcodes.at[location, 'longitude'])
    #except ValueError:
        #house_prices_2019.drop(house_prices_2019[house_prices_2019['Postcode'] == postcode].index, inplace = True)
        
#house_prices_2019['Country'] = country
#house_prices_2019['Latitude'] = latitude
#house_prices_2019['Longitude'] = longitude
#house_prices_2019.to_csv('house_prices_2019.csv', index = False)

#country = []
#latitude = []
#longitude = []
#for postcode in house_prices_2020['Postcode']:
    #try:
        #location = postcodeList.index(postcode)
        #country.append(postcodes.at[location, 'country'])
        #latitude.append(postcodes.at[location, 'latitude'])
        #longitude.append(postcodes.at[location, 'longitude'])
    #except ValueError:
        #house_prices_2020.drop(house_prices_2020[house_prices_2020['Postcode'] == postcode].index, inplace = True)
        
#house_prices_2020['Country'] = country
#house_prices_2020['Latitude'] = latitude
#house_prices_2020['Longitude'] = longitude
#house_prices_2020.to_csv('house_prices_2020.csv', index = False)
print("Data combined into csv files with additional location data")

## Clean data

In [ ]:
house_prices_2019 = pd.read_csv('../input/england-wales-property-sales-locations/england_Wales_2019.csv')
house_prices_wales_2019 = house_prices_2019[house_prices_2019['Country'] == 'Wales'].copy()
# Drop properties of type other ('O'); only interested in residential properties
house_prices_wales_2019.drop(house_prices_wales_2019[house_prices_wales_2019['Property type'] == 'O'].index, inplace = True)
# Replace property types with comprehensible names
house_prices_wales_2019['Property type'] = house_prices_wales_2019['Property type'].replace({'D': 'Detached', 'S': 'Semi-detached', 'T': 'Terraced', 'F': 'Flats'})
house_prices_wales_2019['New'] = house_prices_wales_2019['New'].replace({'N': 'Old', 'Y': 'New'})
house_prices_wales_2019 = house_prices_wales_2019.rename(columns = {'Freehold': 'Tenure'})
house_prices_wales_2019['Tenure'] = house_prices_wales_2019['Tenure'].replace({'F': 'Freehold', 'L': 'Leasehold'})
# Reset index
house_prices_wales_2019 = house_prices_wales_2019.reset_index(drop = True)
house_prices_wales_2019.head()

In [ ]:
house_prices_2020 = pd.read_csv('../input/england-wales-property-sales-locations/england_Wales_2020.csv')
house_prices_wales_2020 = house_prices_2020[house_prices_2020['Country'] == 'Wales'].copy()
# Drop properties of type other ('O'); only interested in residential properties
house_prices_wales_2020.drop(house_prices_wales_2020[house_prices_wales_2020['Property type'] == 'O'].index, inplace = True)
# Replace property types with comprehensible names
house_prices_wales_2020['Property type'] = house_prices_wales_2020['Property type'].replace({'D': 'Detached', 'S': 'Semi-detached', 'T': 'Terraced', 'F': 'Flats'})
house_prices_wales_2020['New'] = house_prices_wales_2020['New'].replace({'N': 'Old', 'Y': 'New'})
house_prices_wales_2020 = house_prices_wales_2020.rename(columns = {'Freehold': 'Tenure'})
house_prices_wales_2020['Tenure'] = house_prices_wales_2020['Tenure'].replace({'F': 'Freehold', 'L': 'Leasehold'})
# Reset index
house_prices_wales_2020 = house_prices_wales_2020.reset_index(drop = True)
house_prices_wales_2020.head()

## Explore data

In [ ]:
# create map of Welsh house sales 2019 using latitude and longitude values
latitude = 52.130660
longitude = -3.7837117
map_wales2019 = folium.Map(location=[latitude, longitude], zoom_start=7)

houselocations = house_prices_wales_2019[['Street', 'Locality', 'Latitude', 'Longitude']].copy()
houselocations.dropna(inplace = True)

# add markers to map
for lat, lng, street, neighbourhood in zip(houselocations['Latitude'], houselocations['Longitude'], houselocations['Street'], houselocations['Locality']):
    label = '{}, {}'.format(street, neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_wales2019) 
    
map_wales2019

In [ ]:
# create map of Welsh house sales 2020 using latitude and longitude values
latitude = 52.1306607
longitude = -3.7837117
map_wales2020 = folium.Map(location=[latitude, longitude], zoom_start=7)

houselocations = house_prices_wales_2020[['Street', 'Locality', 'Latitude', 'Longitude']].copy()
houselocations.dropna(inplace = True)

for lat, lng, street, neighbourhood in zip(houselocations['Latitude'], houselocations['Longitude'], houselocations['Street'], houselocations['Locality']):
    label = '{}, {}'.format(street, neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_wales2020) 
    
map_wales2020

<div class="alert alert-block alert-info" style="margin-top: 20px; color: black;">
Plotting all the property sale locations as points on a map, we see the picture above. The majority of property sales appear to be in clusters at the North and South coasts of Wales. But potentially more interesting may to cluster properties by price and see where different value properties are located spatially.
</div>

### Cluster houses by price

In [ ]:
X = house_prices_wales_2019[['Price']].copy()

clusterNum = 3
k_means = KMeans(init = "k-means++", n_clusters = clusterNum, n_init = 12, random_state = 42)
k_means.fit(X)
house_prices_wales_2019['Cluster labels'] = k_means.labels_
house_prices_wales_2019.head(10)

In [ ]:
X = house_prices_wales_2020[['Price']].copy()

clusterNum = 3
k_means = KMeans(init = "k-means++", n_clusters = clusterNum, n_init = 12, random_state = 42)
k_means.fit(X)
house_prices_wales_2020['Cluster labels'] = k_means.labels_
house_prices_wales_2020.head(10)

In [ ]:
# create map of Welsh house sales 2019 using latitude and longitude values
latitude = 52.1306607
longitude = -3.7837117
map_wales2019 = folium.Map(location=[latitude, longitude], zoom_start=7)

houselocations = house_prices_wales_2019[['Street', 'Locality', 'Latitude', 'Longitude', 'Cluster labels']].copy()

colours = ['red', 'blue','gray', 'darkred', 'lightred', 'orange', 'beige', 'green', 'darkgreen', 'lightgreen', 'darkblue', 'lightblue', 'purple', 'darkpurple', 'pink', 'cadetblue', 'lightgray', 'black']

# add markers to map
for lat, lng, street, neighbourhood, cluster in zip(houselocations['Latitude'], houselocations['Longitude'], houselocations['Street'], houselocations['Locality'], houselocations['Cluster labels']):
    label = '{}, {}, cluster {}'.format(street, neighbourhood, cluster)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color=colours[cluster],
        fill=True,
        fill_color=colours[cluster],
        fill_opacity=0.7,
        parse_html=False).add_to(map_wales2019)  
    
map_wales2019

<div class="alert alert-block alert-info" style="margin-top: 20px; color: black;">
The map above shows no obvious difference in sale price between the North and South clusters. But looking more closely, the property cluster represented by the blue dots appears to cluster together in space as well as in price. This suggests that the blue dots cluster is largely composed of urban properties, expecially the large concentration in the south-east of Wales that ncorresponds to Cardiff and surroundings. There are also several small clusters in the north, which are roughly consistent with the string of towns along the North Welsh coast.
</div>

In [ ]:
"""
Adds new pie chart subplot to figure.
"""
def pieArtist(ax, title, group):
    ax.pie(group.count(), labels = group.groups.keys())
    ax.set_title(title)

"""
Draws pie charts of specified categorical features for each of the three clusters.
"""
def clusterVisualiser(feature, house_prices_wales, cluster_labels):
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize = (15, 15))

    pieArtist(ax1, cluster_labels[0], house_prices_wales[house_prices_wales['Cluster labels'] == cluster_labels[0]].groupby([feature])['Price'])
    pieArtist(ax2, cluster_labels[1], house_prices_wales[house_prices_wales['Cluster labels'] == cluster_labels[1]].groupby([feature])['Price'])
    pieArtist(ax3, cluster_labels[2], house_prices_wales[house_prices_wales['Cluster labels'] == cluster_labels[2]].groupby([feature])['Price'])

    plt.show()

# Display proportion of each property type in the different groups
clusterVisualiser('Property type', house_prices_wales_2019, list(house_prices_wales_2019['Cluster labels'].unique()))

<div class="alert alert-block alert-info" style="margin-top: 20px; color: black;">
The pie charts above provide further evidence for the idea that one property cluster represents urban properties. Cluster 1 consists largely of flats, semi-detached and terraced properties, which is consistent with a urban or sub-urban environment. The other two cluster have a large majority of detached properties, which is more consistent with a rural environemnt. It is important to stress that this delineation is probably not perfect, for example cluster 0 has a slightly smaller proportion of detached properties than cluster 2, which suggests that cluster 0 consists of some urban properties, or more so than cluster 2. It is also important to note that I am not applying a clear definition of 'rural' and 'urban', and that the distinction between them is not sharply defined.
</div>

In [ ]:
house_prices_wales_2019['Cluster labels'] = house_prices_wales_2019['Cluster labels'].replace({0: 'Low-value rural', 1: 'Urban', 2: 'High-value rural'})
house_prices_wales_2019.head(10)

In [ ]:
house_prices_wales_2019.groupby(['Cluster labels', 'Property type'])['Price'].mean().plot(kind = "bar")
plt.ylabel("Price")
plt.xlabel("Property groups")
plt.title("Average Property Prices in Wales 2019")
plt.show()

<div class="alert alert-block alert-info" style="margin-top: 20px; color: black;">
As noted earlier, it appears that one property cluster (blue dots on map) tends to be spatially clustered in towns and cities, surrounded by properties in other clusters at the periphery. In the bar graph above, it can be seen that the urban cluster has the lowest average price, followed by the first rural cluster, which has a higher average price, and finally the second rural cluster, which has the highest average sale price.
</div>

In [ ]:
# Display proportion of new-build properties in the different groups
clusterVisualiser('New', house_prices_wales_2019, list(house_prices_wales_2019['Cluster labels'].unique()))

<div class="alert alert-block alert-info" style="margin-top: 20px; color: black;">
The proportion of new properties is similar across both rural clusters, so it appears unlikely that this is the reason for the price difference between them. There is a much lower proportion of new-build properties in towns and cities, so this may be a factor in the lower prices for urban properties.
</div>

In [ ]:
# Display proportion of leasehold and freehold properties in the different groups
clusterVisualiser('Tenure', house_prices_wales_2019, list(house_prices_wales_2019['Cluster labels'].unique()))

<div class="alert alert-block alert-info" style="margin-top: 20px; color: black;">
The proportion of freehold properties is likewise similar across both rural clusters, so this is unlikely to be the reason for the large difference in sale price ether. There is a much higher proportion of leasehold properties in towns and cities, so this may be a factor in the lower prices for urban properties (leasehold properties come with extra ongoing fees (ground rent) and restrictions, as well as the risk of the lease expiring).
</div>

In [ ]:
# create map of Welsh house sales 2020 using latitude and longitude values
latitude = 52.1306607
longitude = -3.7837117
map_wales2020 = folium.Map(location=[latitude, longitude], zoom_start=7)

houselocations = house_prices_wales_2020[['Street', 'Locality', 'Latitude', 'Longitude', 'Cluster labels']].copy()

colours = ['red', 'blue','gray', 'darkred', 'lightred', 'orange', 'beige', 'green', 'darkgreen', 'lightgreen', 'darkblue', 'lightblue', 'purple', 'darkpurple', 'pink', 'cadetblue', 'lightgray', 'black']

# add markers to map
for lat, lng, street, neighbourhood, cluster in zip(houselocations['Latitude'], houselocations['Longitude'], houselocations['Street'], houselocations['Locality'], houselocations['Cluster labels']):
    label = '{}, {}, cluster {}'.format(street, neighbourhood, cluster)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=2,
        popup=label,
        color=colours[cluster],
        fill=True,
        fill_color=colours[cluster],
        fill_opacity=0.7,
        parse_html=False).add_to(map_wales2020)  
    
map_wales2020

<div class="alert alert-block alert-info" style="margin-top: 20px; color: black;">
The map above show a similar clustering of property sale locations to 2019 sales.
</div>

In [ ]:
# Display proportion of each property type in the different groups
clusterVisualiser('Property type', house_prices_wales_2020, list(house_prices_wales_2020['Cluster labels'].unique()))

<div class="alert alert-block alert-info" style="margin-top: 20px; color: black;">
The pie charts show a similar proportion of property types in each property sale price cluster to 2019, so the clusters can be labelled in the same way.
</div>

In [ ]:
house_prices_wales_2020['Cluster labels'] = house_prices_wales_2020['Cluster labels'].replace({0: 'High-value rural', 1: 'Urban', 2: 'Low-value rural'})
house_prices_wales_2020.head(10)

In [ ]:
house_prices_wales_2020.groupby(['Cluster labels', 'Property type'])['Price'].mean().plot(kind = "bar")
plt.ylabel("Price")
plt.xlabel("Property groups")
plt.title("Average Property Prices in Wales 2020")
plt.show()

<div class="alert alert-block alert-info" style="margin-top: 20px; color: black;">
Average sale prices shown in the bar chart above suggest that not only is the price difference between the clustered property sales the same in 2020 as they were in 2019, but the actual average values for corresponding clusters are the same as well.
</div>

In [ ]:
# Display proportion of new-build properties in the different groups
clusterVisualiser('New', house_prices_wales_2020, list(house_prices_wales_2020['Cluster labels'].unique()))

In [ ]:
# Display proportion of leasehold and freehold properties in the different groups
clusterVisualiser('Tenure', house_prices_wales_2020, list(house_prices_wales_2020['Cluster labels'].unique()))

<div class="alert alert-block alert-info" style="margin-top: 20px; color: black;">
The proportion of old and leasehold properties sold are similar in 2020 to 2019, which suggests that the reasons rural property is more valuable than urban property haven't changed in 2020.
</div>

In [ ]:
print("Number of property sales in 2019:", house_prices_wales_2019['Price'].count())
print("Number of property sales in 2020:", house_prices_wales_2020['Price'].count())

<div class="alert alert-block alert-info" style="margin-top: 20px; color: black;">
In 2020, the property sales data follow a similar pattern to 2019. One cluster appears to largely consist of urban properties, while the other 2 clusters represent low and high value rural properties, respectively. The only change appears to be the number of property sales, which is an obvious consequence of the pandemic. In particular, there appears to be very little change between 2019 and 2020 in the average sale price of different property types. This suggests that the pandemic has not affected the underlying factors of the Welsh property market.
</div>

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (15, 8))
fig.suptitle("Average Prices of New and Old Properties in Wales", fontsize = 16)

ax1.bar(house_prices_wales_2019.groupby(['New'])['Price'].groups.keys(), house_prices_wales_2019.groupby(['New'])['Price'].mean(), color = ['r', 'b'])
ax1.set_xlabel("Price")
ax1.set_title("2019")

ax2.bar(house_prices_wales_2020.groupby(['New'])['Price'].groups.keys(), house_prices_wales_2020.groupby(['New'])['Price'].mean(), color = ['r', 'b'])
ax2.set_xlabel("Price")
ax2.set_title("2020")

plt.show()

<div class="alert alert-block alert-info" style="margin-top: 20px; color: black;">
The graph above suggest that another significant feature is the age of a property. New-build properties appear to attract larger sums of money. The graph below shows that the mean price paid isn't significantly different for 2019 and 2020.
</div>

In [ ]:
fig, (ax1, ax2) = plt.subplots(1, 2, figsize = (15, 8))
fig.suptitle("Average Prices of Property Types in Wales", fontsize = 16)

ax1.bar(house_prices_wales_2019.groupby(['Property type'])['Price'].groups.keys(), house_prices_wales_2019.groupby(['Property type'])['Price'].mean(), color = ['r', 'b', 'y', 'k'])
ax1.set_xlabel("Price")
ax1.set_title("2019")

ax2.bar(house_prices_wales_2020.groupby(['Property type'])['Price'].groups.keys(), house_prices_wales_2020.groupby(['Property type'])['Price'].mean(), color = ['r', 'b', 'y', 'k'])
ax2.set_xlabel("Price")
ax2.set_title("2020")

plt.show()

## Conclusions

<div class="alert alert-block alert-info" style="margin-top: 20px; color: black;">
It was shown above that property prices are to a large extent dependent on location, with urban properties largely in one price band, and predominantly rural properties in two other price bands. This correlation, however imperfect, between a property's price and it's environment is surprising and useful for classifying property sales. The difference in sale price is consistent across both years looked at. This suggests that whatever the reason behind this phenomenon, it is long-term and not affected by current events, even a global pandemic that has significantly altered our way of life in several profound and likely permanent ways. The reason why some rural properties are cheaper than others is not clear from the features available in the data used here. In contrast, the price difference between rural and urban properties is likely due, at least in part, to the greater prevalence of detached and freehold properties in the two rural clusters, compared to the urban cluster.

The generally lower prices of urban properties is good news for those who want to live in a Welsh town or city. But it isn't all bad for those looking for rural Welsh properties, some of which have a lower price due to factors which aren't captured by the current dataset. This may be due to other factors affecting desirability, such as appearance/aesthetics, energy usage/insulation, and space/size of property. This means that anyone with a good eye for these more qualitative factors could potentially have a high return on investment, or alternatively find a nice place to live, with a good eye, patience, and willingness to undertake substantial structural refurbishment.
</div>